In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import pandas as pd
from src import preprocess, plot, classify

input_path = '../data/data.csv'

In [3]:
df = preprocess.parse_data(input_path)
display(df.head(1))
df = preprocess.convert_currency(df)
df = preprocess.category_to_number(df)
display(df.head(1))

,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id
0,1,2015-11-09 14:26:51,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,True,0,2015-07-01 23:03:11,MexicoAccount,email68370,ip111778,card184798


,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,amount_convert
0,1,2015-11-09 14:26:51,76,2,530056.0,64800.0,2,98,1,1,1,0,2015-07-01 23:03:11,1,email68370,ip111778,card184798,2823.984


In [4]:
del df['creationdate']
del df['bookingdate']
del df['mail_id']
del df['ip_id']
del df['card_id']

del df['txid']
del df['amount']

df.head(1)

,issuercountrycode,txvariantcode,bin,currencycode,shoppercountrycode,shopperinteraction,simple_journal,cardverificationcodesupplied,cvcresponsecode,accountcode,amount_convert
0,76,2,530056.0,2,98,1,1,1,0,1,2823.984


In [9]:
# df_selected = df[['currencycode', 'txvariantcode', 'issuercountrycode', 'cvcresponsecode', 'accountcode', 'amount_convert','simple_journal']]
df_selected = df[['issuercountrycode', 'currencycode', 'txvariantcode','cvcresponsecode','amount_convert', 'simple_journal']]
# df_selected = df[['issuercountrycode', 'txvariantcode','bin', 'currencycode','shoppercountrycode', 'amount_convert', 'simple_journal']]

In [10]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
X, y = preprocess.split_data_label(df_selected)

print("All features")
classify.tenfold_cv(nb, X,y)
print("All features -- higher threshold")
classify.tenfold_cv(nb, X,y, threshold=0.95)
print("All features -- smote")
classify.tenfold_cv(nb, X,y,True)
print("All features -- smote + higher threshold")
classify.tenfold_cv(nb, X,y,smote_data=True,threshold=0.995)

All features
TP: 42
FP: 1291
FN: 303
TN: 235400


All features -- higher threshold
TP: 18
FP: 950
FN: 327
TN: 235741


All features -- smote
TP: 312
FP: 52187
FN: 33
TN: 184504


All features -- smote + higher threshold
TP: 50
FP: 1529
FN: 295
TN: 235162




In [7]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
X, y = preprocess.split_data_label(df)

print("All features")
classify.tenfold_cv(mnb, X,y)
print("All features -- higher treshold")
classify.tenfold_cv(mnb, X,y, threshold=1.0)
print("All features -- smote")
classify.tenfold_cv(mnb, X,y,True)
print("All features -- smote + higher threshold")
classify.tenfold_cv(mnb, X,y,smote_data=True,threshold=1.0)

All features
TP: 154
FP: 42247
FN: 191
TN: 194444


All features -- higher treshold
TP: 154
FP: 41914
FN: 191
TN: 194777


All features -- smote
TP: 154
FP: 44054
FN: 191
TN: 192637


All features -- smote + higher threshold
TP: 154
FP: 43530
FN: 191
TN: 193161




In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42, max_features='sqrt', max_depth=50)
print("All features")
classify.tenfold_cv(dt, X,y)
print("All features -- smote")
classify.tenfold_cv(dt, X,y,smote_data=True)
print("All features -- smote + higher threshold")
classify.tenfold_cv(dt, X,y,smote_data=True, threshold=0.95)

All features
TP: 36
FP: 266
FN: 309
TN: 236425


All features -- smote
TP: 65
FP: 1477
FN: 280
TN: 235214


All features -- smote + higher threshold
TP: 62
FP: 1341
FN: 283
TN: 235350




In [258]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2

df_selected = df[['issuercountrycode', 'txvariantcode','currencycode','shoppercountrycode', 'amount_convert', 'simple_journal']]
# df_selected = df[['issuercountrycode', 'txvariantcode','currencycode']]
X_train, X_test, y_train, y_test = preprocess.smote_df(df_selected, test_size=0.2)

In [259]:
dt = DecisionTreeClassifier(random_state=42, max_features='sqrt') #, max_leaf_nodes=20)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[[46961   380]
 [   54    13]]


In [237]:
import graphviz 
from sklearn.tree import export_graphviz

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

vals = df.columns.values
features = np.delete(vals, np.where(vals == 'simple_journal'))
labels = ['Settled', 'Chargeback']

dot_data = export_graphviz(dt, out_file=None, 
                    feature_names=features,  
                    class_names=labels,  
                    filled=True, rounded=True,  
                    special_characters=True)  
graph = graphviz.Source(dot_data)
graph

ValueError: Length of feature_names, 10 does not match number of features, 6

In [277]:
from sklearn.feature_selection import chi2, f_classif, SelectPercentile

chi, pvals = chi2(X,y)
result = pd.DataFrame([chi, pvals], columns=features, index=['chi', 'pval'])
result.sort_values('chi', axis=1)
display(result)

# chi, pvals = f_classif(X_train,y_train)
# result = pd.DataFrame([chi, pvals], columns=features, index=['chi', 'pval'])
# result.sort_values('chi', axis=1)
# display(result)

# 'currencycode', 'txvariantcode', 'issuercountrycode', 'cvcresponsecode', 'accountcode'


,issuercountrycode,txvariantcode,bin,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,accountcode,amount_convert
chi,2.832760e+02,1.078860e+02,119627.072383,10.413018,8.369891e+02,0.621573,3.016407,2.038032e+02,3.352954e+02,2.077845e+06
pval,1.451159e-63,2.846906e-25,0.000000,0.001251,4.899935e-184,0.430463,0.082426,3.089831e-46,6.752710e-75,0.000000e+00
